#COMP9312_Project_Q2: k-Core Index

For more information about the project, please refer to the [project specification](https://cgi.cse.unsw.edu.au/~cs9312/24T2/project/). You can edit this file and add anything you like. We will only use the code cell of the `KCore` class for testing. You can add descriptions and some theoretical analysis (e.g. index space, query time complexity, and index time complexity) to this file without creating a separate PDF document.

**Note**: Make sure to **sequentially run all cells in each section** in order to carry over intermediate variables/packages to the next cell.

## 1. Code Template
You need to implement the `KCore` class to support k-core queries for large graphs (e.g. hundreds of millions of vertices and edges). A code template is given below.

The `KCore` class is initialized by a graph `G`. The data structure of `G` will be presented in the next section. The class calls the function `preprocess` to precompute some index structure for `G`. The `query` function has one inputs: `k`. The function outputs the `k`-cores in `G`.

In [ ]:
import networkx as nx

class KCore(object):
    def __init__(self, G):
        self.G = self.convert_to_networkx(G)
        self.core_number = None
        self.core_index = {}
        self.preprocess(self.G)

    def convert_to_networkx(self, G):
        nx_graph = nx.Graph()
        for u in range(len(G.adj_list)):
            for v in G.adj_list[u]:
                nx_graph.add_edge(u, v)
        return nx_graph

    def preprocess(self, G):
        self.core_number = nx.core_number(G)
        
        # Build index for each core number
        for node, core in self.core_number.items():
            if core not in self.core_index:
                self.core_index[core] = []
            self.core_index[core].append(node)
        
        # Sort each k-core sublist
        for k in self.core_index:
            self.core_index[k].sort()

    def query(self, k):
        cores = []
        # Retrieve all vertices with core number >= k
        k_core_nodes = [node for node, core in self.core_number.items() if core >= k]
        
        if k_core_nodes:
            subgraph = self.G.subgraph(k_core_nodes)
            connected_components = list(nx.connected_components(subgraph))
            cores = [list(component) for component in connected_components]
        
        return cores

## 2. Graph Data Structure
The following is the data stucture of the input graph `G`.

In [ ]:
################################################################################
# Do not edit this code cell.
################################################################################

class UndirectedUnweightedGraph(object):
    def __init__(self, edge_list):
        self.adj_list = []
        self.vertex_num = 0
        self.edge_num = 0
        info = True
        for [vertex_u, vertex_v] in edge_list:
            if info:
                info = False
                self.vertex_num = vertex_u
                self.edge_num = vertex_v
                self.adj_list = [list() for _ in range(self.vertex_num)]
            else:
                self.adj_list[vertex_u].append(vertex_v)
                self.adj_list[vertex_v].append(vertex_u)

## 3. How to test your code

### 3.1 Download the sample dataset.

Running the following command will create the **COMP9312-24T2-Project** folder, which contains the files for the three datasets.

> **Cora** (2k vertices) is a real citation graph, **map_BJ_part** (4k vertices) is a real road network for a small area of Beijing, and **map_NY_part** (7k vertices) is a real road network for a small area of New York. For the two road networks, we erase the weight of each edge in the original dataset to generate unweighted graphs for simplicity.

There are three files for each dataset, where ***.graph** includes the graph information and all graph edges (vertex IDs are consecutive and start from 0), ***.query** includes a set of queries for testing. ***.answer** includes the correct answer to each query for your reference.

If the dataset already exists, an error like "*destination path 'COMP9312-24T2-Project' already exists*" will appear.

**NOTE**: We will test the code using different datasets.

In [ ]:
!git clone https://github.com/kevinChnn/COMP9312-24T2-Project

### 3.2 The main function

Our test procedure first loads the graph dataset and the query dataset. Then, it calls the `KCore` class to preprocess the graph. After that, it will run each query and test its efficiency and correctness.

In [ ]:
import time
import numpy as np
import pickle

if __name__ == "__main__":

    print('\n######## Loading the dataset...')
    edge_list = np.loadtxt('./COMP9312-24T2-Project/cora.graph', dtype=int)
    G = UndirectedUnweightedGraph(edge_list)

    print('\n######## Preprocessing the graph...')
    start_preprocessing = time.time()
    KC = KCore(G)
    end_preprocessing = time.time()
    print("Preprocessing time: {:.8f}".format(end_preprocessing-start_preprocessing))

    print('\n######## Query Testing...')
    k_lo = 2 # inclusive
    k_hi = 5 # exclusive
    for k in range(k_lo,k_hi):
        with open(f'./COMP9312-24T2-Project/cora-{k}.core', 'rb') as f:
            correct_cores = pickle.load(f)
        start_query = time.time()
        cores = KC.query(k)
        end_query = time.time()
        if len(cores) != len(correct_cores) or sum(len(core) for core in cores) != sum(len(core) for core in correct_cores):
            print("Query time: {:.8f} | {}-cores : False".format((end_query-start_query), k))
        else:
            test_cores = {frozenset(core) for core in cores}
            print("Query time: {:.8f} | {}-cores : {}".format((end_query-start_query), k, test_cores==correct_cores))